## 패키지 인스톨

In [ ]:
!pip3 install --upgrade pip
!pip3 --disable-pip-version-check install transformers kobert-transformers seqeval
# !pip3 --disable-pip-version-check install transformers kobert-transformers pororo langid demoji ipadic fugashi konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=8dd7f1738225bea1ae71cb5055a6d6e46822b7f100a79e028a0df80e69e74541
  Stored in

## 데이터 로드 및 학습 & 테스트 데이터 생성

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import csv

In [ ]:
result_df = pd.read_csv("/content/drive/MyDrive/DATA/cp2/result_train_data_ner.v3.tsv", delimiter='\t')
result_df


,channel_name_x,sender_nickname,message,id type,labels
0,김위칭,위칭,으어 오늘도 재밌엉ㅎ습니다,celeb,O DATE O O
1,김위칭,위칭,요즘 계속 업뎃중이라 토큰이 자꾸 만료되네요ㅋㅋ,celeb,O O O O O O
2,김위칭,위칭,ㅋㅋㅋㅋ,celeb,O
3,김위칭,위칭,다들 맛난거 드시네요,celeb,O O O
4,김위칭,위칭,위칭이도 먹고시푼데,celeb,O O
...,...,...,...,...,...
499640,평범입니다!,레이지유,와 진짜 하루동안 톡 엄청하셨네ㅋㅋ,fan,O O DATE O O
499641,평범입니다!,레이지유,ㅋㅋㅋ,fan,O
499642,평범입니다!,레이지유,ㅋㅋㅋ맛있었어요 ㅋㅋ 식사는 하셨나요 식사는 잘 챙겨드십쇼,fan,O O O O O O O
499643,평범입니다!,레이지유,저도 점심중입니다 맛있게 드시고 즐거운 시간보내셔요,fan,O O O O O O


In [ ]:
result_df = result_df[(result_df['message'].notna()) & (result_df['labels'].notna())]

result_df['unique_val_labels'] = result_df['labels'].apply(lambda x: set(x.split()))
result_df['unique_len_message'] = result_df['message'].apply(lambda x: len(set(x.split())))
result_df['unique_len_labels'] = result_df['labels'].apply(lambda x: len(set(x.split())))


result_df = result_df[
    (result_df['unique_len_message'] > 1)
         & (result_df['unique_len_labels'] > 2)
]

result_df.reset_index(drop=True, inplace=True)
result_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,channel_name_x,sender_nickname,message,id type,labels,unique_val_labels,unique_len_message,unique_len_labels
0,김위칭,위칭,정말 소중한 한 표가 된 이번 대선,celeb,O O QUANTITY O O O EVENT,"{EVENT, QUANTITY, O}",7,3
1,김위칭,위칭,이제 기차입니다 오늘 사람 많네요 포항갈땐 사람 없었는데ㅜ,celeb,O ARTIFACT O DATE O O CITY O O O,"{DATE, CITY, O, ARTIFACT}",7,4
2,김위칭,위칭,국방부에서 호스팅한다 트수 또,celeb,ORGANIZATION O O PERSON O,"{ORGANIZATION, PERSON, O}",4,3
3,김위칭,위칭,다들 행복한 금요일 밤 되라냥,celeb,O O DATE TIME O,"{DATE, TIME, O}",5,3
4,김위칭,위칭,마리 너무 귀여워요 제 배에 손 얹은 거 좀 보세여,celeb,QUANTITY O O O ANIMAL O ANIMAL O O O O,"{ANIMAL, QUANTITY, O}",10,3
...,...,...,...,...,...,...,...,...
53353,평범입니다!,멕시코사람,오늘에저녁 고구마 바나나 아몬드,fan,DATE O TIME PLANT PLANT PLANT,"{DATE, PLANT, TIME, O}",4,4
53354,평범입니다!,멕시코사람,오늘의 저녁 고구마 미역 브로콜리 바나나,fan,DATE O TIME CIVILIZATION PLANT CIVILIZATION CI...,"{PLANT, O, CIVILIZATION, TIME, DATE}",6,5
53355,평범입니다!,빵서를,쪼아요 오늘저녁 맛밥하시고 낼아침방송에 봐요,fan,O DATE TIME CIVILIZATION O TIME O O,"{DATE, CIVILIZATION, TIME, O}",5,4
53356,평범입니다!,빵서를,지하철뉴스에도 나온거라 평범님 방송에도 웬지 언급될 수도 있다고 생각해요,fan,ARTIFACT O O PERSON O O O O O O O,"{PERSON, O, ARTIFACT}",9,3


In [ ]:
result_df.size

426864

In [ ]:
# result_df.value_counts()

In [ ]:
result_df.reset_index(drop=True, inplace=True)
result_df

,channel_name_x,sender_nickname,message,id type,labels,unique_val_labels,unique_len_message,unique_len_labels
0,김위칭,위칭,정말 소중한 한 표가 된 이번 대선,celeb,O O QUANTITY O O O EVENT,"{EVENT, QUANTITY, O}",7,3
1,김위칭,위칭,이제 기차입니다 오늘 사람 많네요 포항갈땐 사람 없었는데ㅜ,celeb,O ARTIFACT O DATE O O CITY O O O,"{DATE, CITY, O, ARTIFACT}",7,4
2,김위칭,위칭,국방부에서 호스팅한다 트수 또,celeb,ORGANIZATION O O PERSON O,"{ORGANIZATION, PERSON, O}",4,3
3,김위칭,위칭,다들 행복한 금요일 밤 되라냥,celeb,O O DATE TIME O,"{DATE, TIME, O}",5,3
4,김위칭,위칭,마리 너무 귀여워요 제 배에 손 얹은 거 좀 보세여,celeb,QUANTITY O O O ANIMAL O ANIMAL O O O O,"{ANIMAL, QUANTITY, O}",10,3
...,...,...,...,...,...,...,...,...
53353,평범입니다!,멕시코사람,오늘에저녁 고구마 바나나 아몬드,fan,DATE O TIME PLANT PLANT PLANT,"{DATE, PLANT, TIME, O}",4,4
53354,평범입니다!,멕시코사람,오늘의 저녁 고구마 미역 브로콜리 바나나,fan,DATE O TIME CIVILIZATION PLANT CIVILIZATION CI...,"{PLANT, O, CIVILIZATION, TIME, DATE}",6,5
53355,평범입니다!,빵서를,쪼아요 오늘저녁 맛밥하시고 낼아침방송에 봐요,fan,O DATE TIME CIVILIZATION O TIME O O,"{DATE, CIVILIZATION, TIME, O}",5,4
53356,평범입니다!,빵서를,지하철뉴스에도 나온거라 평범님 방송에도 웬지 언급될 수도 있다고 생각해요,fan,ARTIFACT O O PERSON O O O O O O O,"{PERSON, O, ARTIFACT}",9,3


In [ ]:
labels_set = set()
for i, sr in result_df.iterrows():
    labels_set.update(set(sr['labels'].split()))

labels = ['UNK'] + list(labels_set)
labels

['UNK',
 'EVENT',
 'COUNTRY',
 'PLANT',
 'LOC',
 'NORP',
 'ORGANIZATION',
 'OCCUPATION',
 'STUDY_FIELD',
 'O',
 'GPE',
 'CIVILIZATION',
 'DISEASE',
 'TERM',
 'THEORY',
 'ANIMAL',
 'LOCATION',
 'QUANTITY',
 'CITY',
 'ARTIFACT',
 'PERSON',
 'TIME',
 'MATERIAL',
 'DATE']

In [ ]:
with open('/content/drive/MyDrive/DATA/cp2/train_data/labels.txt', 'a') as f:
    for i, _label in enumerate(labels):
        _newline = "\n" if i < len(labels)-1 else ""
        f.write(f'{_label}{_newline}')

In [ ]:
train_df, test_df = train_test_split(result_df,
                                    test_size=0.1,
                                    random_state=0)

In [ ]:
train_df.reset_index(inplace=True)
# train_df[['message', 'labels']].to_csv('/content/drive/MyDrive/DATA/cp2/train_data/train.tsv',sep='\t', quoting=csv.QUOTE_NONE, index=False, header=False)
train_df.to_csv('/content/drive/MyDrive/DATA/cp2/train_data/train.tsv',sep='\t', quoting=csv.QUOTE_NONE, index=False)
train_df

,index,channel_name_x,sender_nickname,message,id type,labels,unique_val_labels,unique_len_message,unique_len_labels
0,51305,🐻 곰돌파 🐻,쿠키영상,앤님 안녕히 주무셨나요 저는 지금 빗소리 들으면서 커피마시며 글 쓰고있습니다 이따 ...,fan,PERSON O O O O O O O CIVILIZATION O O O O O O ...,"{CIVILIZATION, PERSON, O}",16,3
1,20141,🍒체리쉬_문백찬 Group Channel,밝은달,자기는 빗소리 좋아해요 저는 엄청 좋아해요 일할 때 비오면 애들이 더 산만해져서 안...,fan,O O O O O O O O O O O O O O O O O O O O O O O ...,"{STUDY_FIELD, O, ARTIFACT, PERSON, DATE}",74,5
2,6655,담사장 문열어🔥🔥🔥,새담,솔직히 빠질만하죠 저 여자아이돌 때문에 심장 오랜만에 뜀 ㄹㅇ,celeb,O O O O OCCUPATION O TERM O O O,"{TERM, OCCUPATION, O}",9,3
3,30953,🍒체리쉬_설 윤,☁️,윤님도 저녁 든든하게 잘 먹고서 예쁜말 잔뜩 가지고 와주세요,fan,PERSON O TIME O O O O O O O,"{PERSON, TIME, O}",9,3
4,7441,담사장 문열어🔥🔥🔥,교촌치킨,6시45분 버스를타고 현재 길막히는걸보면서 오열하는중,fan,TIME ARTIFACT O O O O,"{TIME, O, ARTIFACT}",5,3
...,...,...,...,...,...,...,...,...,...
48017,45891,초열,초열,셀럽 팬에게】 가장 좋아하는 베라맛은 채팅 꾸욱 클릭 [전체 공개] 후 답변해 보세요,celeb,CIVILIZATION OCCUPATION O O O CIVILIZATION O O...,"{CIVILIZATION, O, OCCUPATION}",13,3
48018,52416,_코히,설탕별,그럼 코히님은 교수님같은건가,fan,O PERSON O OCCUPATION O,"{OCCUPATION, PERSON, O}",3,3
48019,42613,유메아,대학하는과제생,메아님 김장 당해서 고춧가루 버무려지나요 숙성 양고기가 되나요,fan,PERSON O CIVILIZATION O CIVILIZATION O O CIVIL...,"{CIVILIZATION, PERSON, O}",8,3
48020,43567,나브,누님의 엘리트,두부도 맛있죠 몸에도 좋고,fan,CIVILIZATION O O ANIMAL O O,"{CIVILIZATION, ANIMAL, O}",4,3


In [ ]:
test_df.reset_index(inplace=True)
# test_df[['message', 'labels']].to_csv('/content/drive/MyDrive/DATA/cp2/train_data/test.tsv',sep='\t', quoting=csv.QUOTE_NONE, index=False, header=False)
test_df.to_csv('/content/drive/MyDrive/DATA/cp2/train_data/test.tsv',sep='\t', quoting=csv.QUOTE_NONE, index=False)
test_df

,index,channel_name_x,sender_nickname,message,id type,labels,unique_val_labels,unique_len_message,unique_len_labels
0,26660,🍒체리쉬_반이안 Group Channel☆,젤린,이안님두 밥 잘드셨을테니 뽑뽀 가져가셔용,fan,PERSON O CIVILIZATION O O O,"{CIVILIZATION, PERSON, O}",5,3
1,30765,🍒체리쉬_설 윤,수저미,저녁 맛있게 드시구 남은 하루도 잘보내여 항상 생각하죠,fan,TIME O O O DATE O O O O,"{DATE, TIME, O}",8,3
2,43304,이쿠오_,noname8824,닼소3할때 진짜 팝콘갖고와서 봤는데,fan,O QUANTITY O O CIVILIZATION O O,"{CIVILIZATION, QUANTITY, O}",4,3
3,31033,🍒체리쉬_민영보,배나,영보님하고 찬님 방송 기다리구 있어용 하이볼 말아마셔야지,fan,PERSON O PERSON O O O O CIVILIZATION O,"{CIVILIZATION, PERSON, O}",7,3
4,18423,🍒체리쉬_온해빈 Group Channel,수피아,진짜 말을 왜이렇게 예쁘게 해요 오늘 있었던 힘든일들은 아무것도 아닌게 되어버리네요...,fan,O O O O O DATE O O O O O O O CIVILIZATION O O ...,"{DATE, CIVILIZATION, O}",20,3
...,...,...,...,...,...,...,...,...,...
5331,22420,따찌,pw1023,직장인따찌는 11시에 방송을한다,fan,CIVILIZATION ORGANIZATION O TIME O O,"{ORGANIZATION, CIVILIZATION, TIME, O}",3,4
5332,12367,김씨할매와 손주들,김씨할매,오오오오오 폰 두등탁 전 한번사면 엄청 오래써서,celeb,O TERM QUANTITY O QUANTITY O O O,"{TERM, QUANTITY, O}",7,3
5333,25766,🍒체리쉬_설 윤,_탄산민트,뭄무 ㄱㅇㅇ ㅋㅋㅋㅋ 시간내줘서 고마워요 윤님덕이 오후도 힘내겠슴돠 윤님도 힘내요,fan,O O O O O PERSON O TIME O O PERSON O O,"{PERSON, TIME, O}",10,3
5334,47673,유키냥,유키냥,방송은 아마 9시 합방 아브렐 우유 소영님 과몽님 저,celeb,O O TIME O ARTIFACT CIVILIZATION PERSON O PERS...,"{O, ARTIFACT, CIVILIZATION, PERSON, TIME}",9,5


## 참조 영역

In [ ]:
import logging

logger = logging.getLogger(__name__)

def init_logger():
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
init_logger()

In [ ]:
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBert model."""
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
import os
import random
import logging

import torch
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

from transformers import (
    BertConfig,
    DistilBertConfig,
    ElectraConfig,
    ElectraTokenizer,
    BertTokenizer,
    BertForTokenClassification,
    DistilBertForTokenClassification,
    ElectraForTokenClassification
)

MODEL_CLASSES = {
    'kobert': (BertConfig, BertForTokenClassification, KoBertTokenizer),
    'distilkobert': (DistilBertConfig, DistilBertForTokenClassification, KoBertTokenizer),
    'bert': (BertConfig, BertForTokenClassification, BertTokenizer),
    'kobert-lm': (BertConfig, BertForTokenClassification, KoBertTokenizer),
    'koelectra-base': (ElectraConfig, ElectraForTokenClassification, ElectraTokenizer),
    'koelectra-small': (ElectraConfig, ElectraForTokenClassification, ElectraTokenizer),
}

MODEL_PATH_MAP = {
    'kobert': 'monologg/kobert',
    'distilkobert': 'monologg/distilkobert',
    'bert': 'bert-base-multilingual-cased',
    'kobert-lm': 'monologg/kobert-lm',
    'koelectra-base': 'monologg/koelectra-base-discriminator',
    'koelectra-small': 'monologg/koelectra-small-discriminator',
}


def get_test_texts(args):
    texts = []

    _df = pd.read_csv(os.path.join(args.data_dir, args.test_file), delimiter='\t')
    for i, sr in _df.iterrows():
        texts.append(sr['message'])

    # with open(os.path.join(args.data_dir, args.test_file), 'r', encoding='utf-8') as f:
    #     for line in f:
    #         text, _ = line.split('\t')
    #         text = text.split()
    #         texts.append(text)

    return texts


def compute_metrics(labels, preds):
    assert len(preds) == len(labels)
    return f1_pre_rec(labels, preds)


def f1_pre_rec(labels, preds):
    return {
        "precision": precision_score(labels, preds, suffix=True),
        "recall": recall_score(labels, preds, suffix=True),
        "f1": f1_score(labels, preds, suffix=True)
    }


def show_report(labels, preds):
    return classification_report(labels, preds, suffix=True)

In [ ]:
import os
import copy
import json
import logging

import torch
from torch.utils.data import TensorDataset

logger = logging.getLogger(__name__)


class InputExample(object):
    """
    A single training/test example for simple sequence classification.

    Args:
        guid: Unique id for the example.
        words: list. The words of the sequence.
        labels: (Optional) list. The slot labels of the example.
    """

    def __init__(self, guid, words, labels):
        self.guid = guid
        self.words = words
        self.labels = labels

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label_ids = label_ids

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"



In [ ]:
import os
import shutil
import logging
from tqdm import tqdm, trange

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

logger = logging.getLogger(__name__)


class Trainer(object):
    def __init__(self, args, train_dataset=None, dev_dataset=None, test_dataset=None):
        self.args = args
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset

        self.label_lst = get_labels(args)
        self.num_labels = len(self.label_lst)
        # Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
        self.pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index

        self.config_class, self.model_class, _ = MODEL_CLASSES[args.model_type]

        self.config = self.config_class.from_pretrained(args.model_name_or_path,
                                                        num_labels=self.num_labels,
                                                        finetuning_task=args.task,
                                                        id2label={str(i): label for i, label in enumerate(self.label_lst)},
                                                        label2id={label: i for i, label in enumerate(self.label_lst)})
        self.model = self.model_class.from_pretrained(args.model_name_or_path, config=self.config)

        
        if "tpu" == args.device.lower():
            # TPU
            import torch_xla.core.xla_model as xm
            self.device = xm.xla_device()
            xm.set_rng_state(args.seed, self.device)
            
        else:
            # GPU or CPU
            self.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
        
        self.model.to(self.device)

        self.test_texts = None
        if args.write_pred:
            self.test_texts = get_test_texts(args)
            # Empty the original prediction files
            if os.path.exists(args.pred_dir):
                shutil.rmtree(args.pred_dir)

    def train(self):
        train_sampler = RandomSampler(self.train_dataset)
        train_dataloader = DataLoader(self.train_dataset, sampler=train_sampler, batch_size=self.args.train_batch_size)

        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.args.num_train_epochs = self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
        else:
            t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': self.args.weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=t_total)

        # Train!
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(self.train_dataset))
        logger.info("  Num Epochs = %d", self.args.num_train_epochs)
        logger.info("  Total train batch size = %d", self.args.train_batch_size)
        logger.info("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
        logger.info("  Total optimization steps = %d", t_total)
        logger.info("  Logging steps = %d", self.args.logging_steps)
        logger.info("  Save steps = %d", self.args.save_steps)

        global_step = 0
        tr_loss = 0.0
        self.model.zero_grad()

        train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch")

        for _ in train_iterator:
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                self.model.train()
                batch = tuple(t.to(self.device) for t in batch)  # GPU or CPU
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'labels': batch[3]}
                if self.args.model_type != 'distilkobert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                loss = outputs[0]

                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                loss.backward()

                tr_loss += loss.item()
                if (step + 1) % self.args.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()
                    global_step += 1

                    if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
                        self.evaluate("test", global_step)

                    if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
                        self.save_model()

                if 0 < self.args.max_steps < global_step:
                    epoch_iterator.close()
                    break

            if 0 < self.args.max_steps < global_step:
                train_iterator.close()
                break

        return global_step, tr_loss / global_step

    def evaluate(self, mode, step):
        if mode == 'test':
            dataset = self.test_dataset
        elif mode == 'dev':
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        logger.info("***** Running evaluation on %s dataset *****", mode)
        logger.info("  Num examples = %d", len(dataset))
        logger.info("  Batch size = %d", self.args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None

        self.model.eval()

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'labels': batch[3]}
                if self.args.model_type != 'distilkobert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

            # Slot prediction
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs["labels"].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        results = {
            "loss": eval_loss
        }

        # Slot result
        preds = np.argmax(preds, axis=2)
        slot_label_map = {i: label for i, label in enumerate(self.label_lst)}
        out_label_list = [[] for _ in range(out_label_ids.shape[0])]
        preds_list = [[] for _ in range(out_label_ids.shape[0])]

        for i in range(out_label_ids.shape[0]):
            for j in range(out_label_ids.shape[1]):
                if out_label_ids[i, j] != self.pad_token_label_id:
                    out_label_list[i].append(slot_label_map[out_label_ids[i][j]])
                    preds_list[i].append(slot_label_map[preds[i][j]])

        if self.args.write_pred:
            if not os.path.exists(self.args.pred_dir):
                os.mkdir(self.args.pred_dir)

            with open(os.path.join(self.args.pred_dir, "pred_{}.txt".format(step)), "w", encoding="utf-8") as f:
                for text, true_label, pred_label in zip(self.test_texts, out_label_list, preds_list):
                    for t, tl, pl in zip(text, true_label, pred_label):
                        f.write("{} {} {}\n".format(t, tl, pl))
                    f.write("\n")

        result = compute_metrics(out_label_list, preds_list)
        results.update(result)

        logger.info("***** Eval results *****")
        for key in sorted(results.keys()):
            logger.info("  %s = %s", key, str(results[key]))
        logger.info("\n" + show_report(out_label_list, preds_list))  # Get the report for each tag result

        return results

    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.args.model_dir):
            os.makedirs(self.args.model_dir)
        model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        model_to_save.save_pretrained(self.args.model_dir)

        # Save training arguments together with the trained model
        torch.save(self.args, os.path.join(self.args.model_dir, 'training_args.bin'))
        logger.info("Saving model checkpoint to %s", self.args.model_dir)

    def load_model(self):
        # Check whether model exists
        if not os.path.exists(self.args.model_dir):
            raise Exception("Model doesn't exists! Train first!")

        try:
            self.model = self.model_class.from_pretrained(self.args.model_dir)
            self.model.to(self.device)
            logger.info("***** Model Loaded *****")
        except:
            raise Exception("Some model files might be missing...")

In [ ]:
# print(f"words:{len(words)}, labels:{len(labels_idx)}")
# assert len(words) == len(labels_idx)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def get_labels(args):
    return [label.strip() for label in open(os.path.join(args.data_dir, args.label_file), 'r', encoding='utf-8')]

class FanSimProcessor(object):
    """Processor for the Naver NER data set """

    def __init__(self, args):
        self.args = args
        self.labels_lst = get_labels(args)

    @classmethod
    def _read_file(cls, input_file):
        """Read tsv file, and return words and label as list"""
        return pd.read_csv(input_file, delimiter='\t')
        # with open(input_file, "r", encoding="utf-8") as f:
        #     lines = []
        #     for line in f:
        #         lines.append(line.strip())

        #     return lines

    def _create_examples(self, dataset, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        i = 0
        # for data in dataset:
        #     words, labels = data.split('\t')
        #     words = words.split()
        #     labels = labels.split()

        #     labels_idx = []
        #     for label in labels:
        #         labels_idx.append(self.labels_lst.index(label) if label in self.labels_lst else self.labels_lst.index("UNK"))
            
        #     # print(data)
        #     # print(f"words:{len(words)}, labels:{len(labels_idx)}")
        #     # assert len(words) == len(labels_idx)
        #     if len(words) != len(labels_idx):
        #         continue

        #     # if i % 10000 == 0:
        #     #     print(data)
        #     examples.append(InputExample(guid=f"{set_type}-{i}", words=words, labels=labels_idx))
        #     i += 1
            
        for j, sr in dataset.iterrows():
            # words, labels = data.split('\t')
            # words = words.split()
            # labels = labels.split()
            words = sr['message'].split()
            labels = sr['labels'].split()
            
            labels_idx = []
            for label in labels:
                labels_idx.append(self.labels_lst.index(label) if label in self.labels_lst else self.labels_lst.index("UNK"))
            
            # print(data)
            # print(f"words:{len(words)}, labels:{len(labels_idx)}")
            # assert len(words) == len(labels_idx)
            if len(words) != len(labels_idx):
                continue

            # if i % 10000 == 0:
            #     print(data)
            examples.append(InputExample(guid=f"{set_type}-{i}", words=words, labels=labels_idx))
            i += 1
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, dev, test
        """
        file_to_read = None
        if mode == 'train':
            file_to_read = self.args.train_file
        elif mode == 'dev':
            file_to_read = self.args.dev_file
        elif mode == 'test':
            file_to_read = self.args.test_file

        print("LOOKING AT {}".format(os.path.join(self.args.data_dir, file_to_read)))
        return self._create_examples(self._read_file(os.path.join(self.args.data_dir, file_to_read)), mode)

train_dataset = None
dev_dataset = None
test_dataset = None

def convert_examples_to_features(examples, max_seq_len, tokenizer,
                                 pad_token_label_id=-100,
                                 cls_token_segment_id=0,
                                 pad_token_segment_id=0,
                                 sequence_a_segment_id=0,
                                 mask_padding_with_zero=True):
    # Setting based on the current model type
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    unk_token = tokenizer.unk_token
    pad_token_id = tokenizer.pad_token_id

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 5000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        # Tokenize word by word (for NER)
        tokens = []
        label_ids = []
        for word, slot_label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            if not word_tokens:
                word_tokens = [unk_token]  # For handling the bad-encoded word
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([int(slot_label)] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP]
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[: (max_seq_len - special_tokens_count)]
            label_ids = label_ids[: (max_seq_len - special_tokens_count)]

        # Add [SEP] token
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        token_type_ids = [sequence_a_segment_id] * len(tokens)

        # Add [CLS] token
        tokens = [cls_token] + tokens
        label_ids = [pad_token_label_id] + label_ids
        token_type_ids = [cls_token_segment_id] + token_type_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_len - len(input_ids)
        input_ids = input_ids + ([pad_token_id] * padding_length)
        attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        token_type_ids = token_type_ids + ([pad_token_segment_id] * padding_length)
        label_ids = label_ids + ([pad_token_label_id] * padding_length)

        assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_ids) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_ids), max_seq_len)
        assert len(label_ids) == max_seq_len, "Error with slot labels length {} vs {}".format(len(label_ids), max_seq_len)

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % example.guid)
            logger.info("tokens: %s" % " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("attention_mask: %s" % " ".join([str(x) for x in attention_mask]))
            logger.info("token_type_ids: %s" % " ".join([str(x) for x in token_type_ids]))
            logger.info("label: %s " % " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids,
                          label_ids=label_ids
                          ))

    return features

def load_and_cache_examples(args, tokenizer, mode):
    processor = FanSimProcessor(args)

    # Load data features from cache or dataset file
    cached_file_name = 'cached_{}_{}_{}_{}'.format(
        args.task, list(filter(None, args.model_name_or_path.split("/"))).pop(), args.max_seq_len, mode)

    pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index
    cached_features_file = os.path.join(args.data_dir, cached_file_name)
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        if mode == "train":
            examples = processor.get_examples("train")
        elif mode == "dev":
            examples = processor.get_examples("dev")
        elif mode == "test":
            examples = processor.get_examples("test")
        else:
            raise Exception("For mode, Only train, dev, test is available")

        features = convert_examples_to_features(examples, args.max_seq_len, tokenizer, pad_token_label_id=pad_token_label_id)
        print("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_label_ids)
    return dataset

## 실행 영역

In [ ]:
class setting:
    device = "tpu" # cpu, gpu, tpu
    seed = 42
    no_cuda = True
    model_type = "kobert"
    model_name_or_path = None
    do_train = True
    do_eval = True
    task = "fansim-ner"
    max_seq_len = 50
    data_dir = "/content/drive/MyDrive/DATA/cp2/train_data"
    model_dir = "/content/drive/MyDrive/MODEL/cp2/model"
    pred_dir = "/content/drive/MyDrive/MODEL/cp2/preds"

    train_file = "train.tsv"
    test_file = "test.tsv"
    label_file = "labels.txt"

    write_pred = True
    
    # train_batch_size = 32
    train_batch_size = 128
    # eval_batch_size = 64
    eval_batch_size = 256
    learning_rate = 5e-5
    num_train_epochs = 3.0
    weight_decay = 0.0
    gradient_accumulation_steps = 1
    adam_epsilon = 1e-8
    max_grad_norm = 1.0
    max_steps = 500
    warmup_steps = 0

    logging_steps = 1000
    save_steps = 1000

setting.model_name_or_path = MODEL_PATH_MAP[setting.model_type]

In [ ]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

def load_tokenizer(args):
    return MODEL_CLASSES[args.model_type][2].from_pretrained(args.model_name_or_path)


set_seed(setting)

tokenizer = load_tokenizer(setting)


Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
dev_dataset = None
test_dataset = load_and_cache_examples(setting, tokenizer, mode="test")

In [ ]:
train_dataset = load_and_cache_examples(setting, tokenizer, mode="train")

In [ ]:
trainer = Trainer(setting, train_dataset, dev_dataset, test_dataset)

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2935 [00:00<?, ?it/s]

Iteration:   0%|          | 1/2935 [00:16<13:03:18, 16.02s/it]

Iteration:   0%|          | 2/2935 [01:31<41:30:07, 50.94s/it]

Iteration:   0%|          | 3/2935 [04:25<87:11:39, 107.06s/it]

Iteration:   0%|          | 4/2935 [09:36<152:53:35, 187.79s/it]

In [ ]:
trainer.load_model()
trainer.evaluate("test", "eval")